In [0]:
from delta.tables import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window
stageDeltaTable = 'spark_workspace.default.stage_table'
ingestTable = 'spark_workspace.default.ingest_table'



In [0]:
winSpec = Window.partitionBy('order_num').orderBy(col('last_update_timestamp').desc())

stage_df = spark.read.table(stageDeltaTable)

stageDf_rn = stage_df.withColumn('row_number',row_number().over(winSpec))


stageDf_Filtered =  stageDf_rn[stageDf_rn['row_number']==1]

if spark._jsparkSession.catalog().tableExists(ingestTable):
    target_table = DeltaTable.forName(spark, ingestTable)

    # Define the merge condition based on the tracking_num column
    merge_condition = "stage.tracking_num = target.tracking_num"

    # Execute the merge operation for SCD Type 2
    target_table.alias("target") \
        .merge(stageDf_Filtered.alias("stage"), merge_condition) \
        .whenMatchedUpdate(set={
            "target.status": "stage.status",
            "target.last_update_timestamp": "stage.last_update_timestamp"
        }) \
        .whenNotMatchedInsert(values={
            "target.order_num": "stage.order_num",
            "target.tracking_num": "stage.tracking_num",
            "target.pck_recieved_date": "stage.pck_recieved_date",
            "target.package_deliver_date": "stage.package_deliver_date",
            "target.status": "stage.status",
            "target.address": "stage.address",
            "target.last_update_timestamp":"stage.last_update_timestamp"
        }) \
        .execute()

    # stage_df.write.format("delta").mode("append").saveAsTable(ingestTable)
else:
    stage_df.write.format("delta").saveAsTable(ingestTable)